In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
pip install -q split-folders

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D,Dense,Flatten,Activation,MaxPooling2D,InputLayer,GlobalMaxPool2D,Dropout
from warnings import filterwarnings
import tensorflow.keras.backend as k
from tensorflow.keras.optimizers import Adam
filterwarnings('ignore')
from sklearn.metrics import classification_report

In [5]:
import splitfolders  # or import split_folders

# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio("../input/surface-crack-detection", output="output1", seed=1337, ratio=(.8, .1, .1), group_prefix=None)

In [6]:
train_data='./output1/train'
val_data='./output1/val'
test_data='./output1/test'

In [7]:
IMAGE_SIZE=224
BATCH_SIZE=32

In [8]:
train_gen=ImageDataGenerator(rescale=1./255,
                            horizontal_flip=True,
                            rotation_range=20,
                            brightness_range=(0,0.2),
                            zoom_range=0.35)
train=train_gen.flow_from_directory(train_data,
                                    color_mode='rgb',
                                    target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                    class_mode='sparse',
                                    batch_size=BATCH_SIZE
                                   )

In [9]:
val_gen=ImageDataGenerator(rescale=1/.255,
                            horizontal_flip=True,
                            rotation_range=20,
                            brightness_range=(0,0.2),
                            zoom_range=0.35)
val=val_gen.flow_from_directory(val_data,
                                    color_mode='rgb',
                                    target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                    class_mode='sparse',
                                    batch_size=BATCH_SIZE
                                   )

In [10]:
test_gen=ImageDataGenerator(rescale=1/.255)
                            
test=test_gen.flow_from_directory(test_data,
                                    color_mode='rgb',
                                    target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                    class_mode='sparse',
                                    batch_size=BATCH_SIZE
                                   )

In [11]:
train.class_indices

In [12]:
classes=list(train.class_indices.keys())
classes

In [13]:
for image,label in train:
    print(train[0])
    break

In [14]:
for image,label in train:
    print(label)
    break

In [15]:
for image in test:
    print(train[0])
    break

In [60]:
#input_shape=(24, 200, 200, 3)
#24 is batch size

model=models.Sequential([
    InputLayer(input_shape=(200,200,3)),
    Conv2D(32,kernel_size=(3,3),padding='same',activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(64,kernel_size=(3,3),padding='same',activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(64,kernel_size=(3,3),padding='same',activation='relu'),
    Dropout(0.2),
    MaxPooling2D((2,2)),
    GlobalMaxPool2D(),
    Flatten(),
    Dense(32,activation='relu'),
    Dense(2,activation='softmax')
])

In [61]:
model.summary()

In [62]:
from tensorflow.keras.utils import plot_model
plot_model(model,to_file='Surface Crack Detection.jpg',show_shapes=True,show_dtype=True,show_layer_names=True,dpi=400)

In [59]:
k.clear_session()

In [63]:
model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
)

In [64]:
len(train)//32

In [65]:
len(val)

In [66]:
hist=model.fit(train,batch_size=BATCH_SIZE,epochs=25,steps_per_epoch=31,validation_data=val,validation_steps=15)

In [67]:
r=model.evaluate(test)

In [68]:
y_pred=model.predict(test)

In [69]:
y=np.argmax(y_pred,axis=1)

In [70]:
y[:20]

In [71]:
import pandas as pd
def histofunc(hist): 
    pd.DataFrame(hist.history).plot(figsize=(12,9))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.xlabel('Epochs',fontsize=18)
    plt.ylabel('Parameters Range',fontsize=18)
    plt.title('Model Performance',fontsize=20)
    return plt.show()
histofunc(hist)

In [72]:
import tensorflow as tf
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i])
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = classes[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [73]:
plt.figure(figsize=(15, 15))
for images, labels in test:
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        
        predicted_class, confidence = predict(model, images[i])
        actual_class = classes[int(labels[i])] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")
    break

In [77]:
target_names = []

for key in train.class_indices:

    target_names.append(key)



print(target_names)

In [91]:
model.save('./ocrack.h5')

**DEMO DEPLOY OF MODEL**

In [92]:
pip install gradio

In [95]:
def predict_image(img):
    imgs=img.reshape(-1,224,224,3)
    prediction=model.predict(imgs)[0]
    return {labels[i]: float(prediction[i]) for i in range(2)}

image = gr.inputs.Image(shape=(224,224))
label = gr.outputs.Label(num_top_classes=2)

gr.Interface(fn=predict_image, inputs=image, outputs=label,interpretation='default',title='Surface Crack Detection',allow_screenshot=True).launch()